In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from backend.data_processing_service import DataProcessingService

# Initialize DataProcessingService
data_processor = DataProcessingService(seq_length=10, pred_window=1, batch_size=10)
data = np.loadtxt('data/normalized_apple_prices.csv')
# data = data_processor.get_sample_data(length=50)

# Normalize the data
data_normalized, scaler = data_processor.normalize_data(data)

# Create sequences
x_data, y_data = data_processor.create_sequences(data_normalized)

# Split data into training and test sets
x_train, x_test, y_train, y_test = data_processor.split_data(x_data, y_data, train_ratio=0.8)

# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [3]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

torch.Size([102, 10, 1]) torch.Size([102, 1, 1]) torch.Size([26, 10, 1]) torch.Size([26, 1, 1])


In [4]:
from backend.model_service import LinRegNN

# Initialize the model
input_size = data_processor.seq_length
output_size = data_processor.pred_window
model = LinRegNN(input_size, output_size)


# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
x_train_rs,x_test_rs= model.reshape_input(x_train),model.reshape_input(x_test)
epochs = 10
model.train_loop(model, criterion, optimizer, x_train_rs, y_train, epochs, directory = "backend/checkpoints/lr01_")
# Eval
model.evaluation(model, x_test_rs, y_test, criterion)

Epoch [1/10], Loss: 0.4989
Epoch [2/10], Loss: 0.2577
Epoch [3/10], Loss: 0.1651
Epoch [4/10], Loss: 0.1311
Epoch [5/10], Loss: 0.1121
Checkpoint saved at epoch 5
Epoch [6/10], Loss: 0.0971
Epoch [7/10], Loss: 0.0842
Epoch [8/10], Loss: 0.0730
Epoch [9/10], Loss: 0.0634
Epoch [10/10], Loss: 0.0552
Checkpoint saved at epoch 10
Test Loss: 0.0176


0.017637882589462188